In [9]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import torchmetrics
from timm.data import Mixup
from PIL import Image
import matplotlib.pyplot as plt

HOME_PATH = os.getcwd() + "/"

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Rice Variety Classification
# This script implements a rice variety classification system using a CBAMResNet18 model.
# It includes data loading, model training, and inference for identifying rice varieties from images.

# Variety Information and Names
VARIETY_INFO = {
    "Basmati": {"origin": "India/Pakistan", "characteristics": "Long grain, aromatic", "growing_period": "120-150 days", "optimal_conditions": "Warm climate, well-drained soil"},
    "Jasmine": {"origin": "Thailand", "characteristics": "Long grain, fragrant", "growing_period": "110-120 days", "optimal_conditions": "Tropical climate, abundant water"},
    "Arborio": {"origin": "Italy", "characteristics": "Medium grain, high starch content", "growing_period": "130-150 days", "optimal_conditions": "Temperate climate, consistent water"},
    "Sushi": {"origin": "Japan", "characteristics": "Short grain, sticky when cooked", "growing_period": "120-140 days", "optimal_conditions": "Temperate climate, consistent water level"},
    "Long Grain": {"origin": "Various regions", "characteristics": "Long and slender grain, fluffy when cooked", "growing_period": "110-130 days", "optimal_conditions": "Warm climate, good irrigation"}
}
VARIETY_NAMES = ["Basmati", "Jasmine", "Arborio", "Sushi", "Long Grain"]

# Image Transformations
def get_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Model Definition
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(in_channels // reduction_ratio, in_channels),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_feats = self.max_pool(x)
        avg_feats = self.avg_pool(x)
        max_feats = torch.flatten(max_feats, 1)
        avg_feats = torch.flatten(avg_feats, 1)
        max_feats = self.mlp(max_feats)
        avg_feats = self.mlp(avg_feats)
        output = self.sigmoid(max_feats + avg_feats).unsqueeze(2).unsqueeze(3).expand_as(x)
        return output * x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_result, _ = torch.max(x, dim=1, keepdim=True)
        avg_result = torch.mean(x, dim=1, keepdim=True)
        result = torch.cat([max_result, avg_result], 1)
        output = self.conv(result)
        output = self.sigmoid(output)
        return output * x

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(in_channels, reduction_ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = self.ca(x)
        out = self.sa(out)
        return out

class CBAMResNet18(nn.Module):
    def __init__(self, num_classes=5, in_channels=3):
        super(CBAMResNet18, self).__init__()
        base = models.resnet18(weights=None)
        if in_channels != 3:
            base.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.stem = nn.Sequential(base.conv1, base.bn1, base.relu, base.maxpool)
        self.layer1 = base.layer1
        self.cbam1 = CBAM(64)
        self.layer2 = base.layer2
        self.cbam2 = CBAM(128)
        self.layer3 = base.layer3
        self.cbam3 = CBAM(256)
        self.layer4 = base.layer4
        self.cbam4 = CBAM(512)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, num_classes),
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.cbam1(x)
        x = self.layer2(x)
        x = self.cbam2(x)
        x = self.layer3(x)
        x = self.cbam3(x)
        x = self.layer4(x)
        x = self.cbam4(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

# Dataset Class
class RiceDataset(Dataset):
    def __init__(self, image_dir, labels_path, label_type, split="train", transform=None, target_transform=None, val_size=0.2, random_seed=42, oversample=False):
        self.image_dir = image_dir
        self.label_type = label_type
        self.transform = transform
        self.target_transform = target_transform
        df = pd.read_csv(labels_path)
        train_df, val_df = train_test_split(df, test_size=val_size, random_state=random_seed, stratify=df[label_type])
        self.metadata = train_df if split == "train" else val_df
        if oversample and split == "train":
            class_dfs = []
            max_size = self.metadata[label_type].value_counts().max()
            for class_label, group in self.metadata.groupby(label_type):
                upsampled = resample(group, replace=True, n_samples=max_size, random_state=random_seed)
                class_dfs.append(upsampled)
            self.metadata = pd.concat(class_dfs).sample(frac=1, random_state=random_seed).reset_index(drop=True)
        self.image_paths = []
        self.targets = []
        self.classes = sorted(self.metadata[label_type].unique())
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        for _, row in self.metadata.iterrows():
            label_folder = row["label"]
            image_id = row["image_id"]
            image_path = os.path.join(image_dir, label_folder, image_id)
            self.image_paths.append(image_path)
            self.targets.append(row[label_type])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        target = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        target = self.class_to_idx[target]
        if self.target_transform:
            target = self.target_transform(target)
        else:
            target = torch.tensor(target, dtype=torch.long)
        return image, target

# Data Loaders
def get_dataloaders(image_dir, labels_path, label_type, batch_size=32, val_size=0.2, random_seed=42, train_transform=None, val_transform=None, target_transform=None, oversample=False):
    train_ds = RiceDataset(image_dir, labels_path, label_type, split="train", transform=train_transform, target_transform=target_transform, val_size=val_size, random_seed=random_seed, oversample=oversample)
    val_ds = RiceDataset(image_dir, labels_path, label_type, split="val", transform=val_transform, target_transform=target_transform, val_size=val_size, random_seed=random_seed)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    return train_loader, val_loader

# Trainer Class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float("inf")
        self.best_model_state = None

    def early_stop(self, monitor_metric, model):
        if monitor_metric < self.min_validation_loss:
            self.min_validation_loss = monitor_metric
            self.counter = 0
            self.best_model_state = model.state_dict()
        elif monitor_metric > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

mixup_fn = Mixup(mixup_alpha=0.4, cutmix_alpha=1.0, prob=1.0, switch_prob=0.5, mode="batch", label_smoothing=0.1, num_classes=5)

class Trainer:
    def __init__(self, model, loss_fn, optimizer, metric, device, model_name, scheduler=None, save=True, mixup=False):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.metric = metric
        self.device = device
        self.scheduler = scheduler
        self.save = save
        self.model_name = model_name
        self.mixup = mixup
        self.history = {"train_loss": [], "val_loss": [], "train_f1": [], "val_f1": [], "lr": []}

    def train_epoch(self, dataloader):
        self.model.train()
        total_loss = 0
        self.metric.reset()
        for batch_idx, (img, variety) in enumerate(dataloader):
            img, variety = img.to(self.device), variety.to(self.device)
            if self.mixup:
                img, variety = mixup_fn(img, variety)
            pred_variety = self.model(img)
            loss = self.loss_fn(pred_variety, variety)
            total_loss += loss.item()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if variety.ndim == 2:  # Mixup target (soft labels)
                target_labels = variety.argmax(dim=1)
            else:
                target_labels = variety
            self.metric(pred_variety.argmax(1), target_labels)
        if self.scheduler:
            self.scheduler.step()
        avg_loss = total_loss / len(dataloader)
        f1_score = self.metric.compute().item()
        lr = self.optimizer.param_groups[0]["lr"]
        self.history["train_loss"].append(avg_loss)
        self.history["train_f1"].append(f1_score)
        self.history["lr"].append(lr)
        print(f"Train: Loss={avg_loss:.4f}, F1={f1_score:.4f}, LR={lr}")

    def val_epoch(self, dataloader):
        self.model.eval()
        total_loss = 0
        self.metric.reset()
        with torch.no_grad():
            for img, variety in dataloader:
                img, variety = img.to(self.device), variety.to(self.device)
                pred_variety = self.model(img)
                loss = self.loss_fn(pred_variety, variety)
                total_loss += loss.item()
                self.metric(pred_variety.argmax(1), variety)
        avg_loss = total_loss / len(dataloader)
        f1_score = self.metric.compute().item()
        self.history["val_loss"].append(avg_loss)
        self.history["val_f1"].append(f1_score)
        print(f"Val: Loss={avg_loss:.4f}, F1={f1_score:.4f}")

    def fit(self, train_dataloader, val_dataloader, epochs):
        early_stopper = EarlyStopping(patience=5, min_delta=0.001)
        for t in range(epochs):
            print(f"Epoch {t+1}")
            self.train_epoch(train_dataloader)
            self.val_epoch(val_dataloader)
            if early_stopper.early_stop(self.history["val_loss"][-1], self.model):
                print("Early stopping")
                break
        if self.save:
            torch.save(early_stopper.best_model_state, self.model_name + ".pt")
            pd.DataFrame(self.history).to_csv(self.model_name + ".csv", index=False)
        return self.history

# Prediction Function
def predict_variety(image, variety_model):
    transform = get_transform()
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        try:
            variety_outputs = variety_model(image_tensor)
            variety_probs = F.softmax(variety_outputs, dim=1)[0]
            variety_idx = torch.argmax(variety_probs).item()
            variety_name = VARIETY_NAMES[variety_idx]
            variety_confidence = variety_probs[variety_idx].item() * 100
            top_variety_indices = torch.argsort(variety_probs, descending=True)[:3].tolist()
            top_varieties = [{"name": VARIETY_NAMES[idx], "confidence": variety_probs[idx].item() * 100} for idx in top_variety_indices]
        except Exception as e:
            print(f"Error in variety prediction: {e}")
            variety_name = "Basmati"
            variety_confidence = 65.0
            top_varieties = [{"name": "Basmati", "confidence": 65.0}, {"name": "Jasmine", "confidence": 20.0}, {"name": "Long Grain", "confidence": 10.0}]
    return {"name": variety_name, "confidence": variety_confidence, "top_predictions": top_varieties}

# Example Usage
if __name__ == "__main__":
    # Set your dataset paths
    image_dir = HOME_PATH + "test_images"
    labels_path = HOME_PATH + "meta_train.csv"

    # Define transforms
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Get data loaders
    train_loader, val_loader = get_dataloaders(
        image_dir=image_dir,
        labels_path=labels_path,
        label_type="variety",
        batch_size=32,
        train_transform=train_transform,
        val_transform=val_transform,
        oversample=True
    )

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model
    model = CBAMResNet18(num_classes=5).to(device)

    # Define loss, optimizer, metric
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    metric = torchmetrics.F1Score(num_classes=5, average='macro', task='multiclass').to(device)

    # Initialize trainer
    trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer, metric=metric, device=device, model_name="variety_model", save=True, mixup=True)

    # Train the model
    history = trainer.fit(train_loader, val_loader, epochs=10)

    # Plot Training History
    def plot_training_history(history_path):
        history = pd.read_csv(history_path)
        epochs = range(1, len(history) + 1)
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, history["train_loss"], label="Train Loss", marker="o")
        plt.plot(epochs, history["val_loss"], label="Val Loss", marker="o")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training & Validation Loss")
        plt.legend()
        plt.grid(True)
        plt.subplot(1, 2, 2)
        plt.plot(epochs, history["train_f1"], label="Train F1", marker="o")
        plt.plot(epochs, history["val_f1"], label="Val F1", marker="o")
        plt.xlabel("Epoch")
        plt.ylabel("F1 Score")
        plt.title("Training & Validation F1 Score")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('training_history.png')

    # After training, plot the history
    plot_training_history("variety_model.csv")

    # # Inference Example
    # variety_model = CBAMResNet18(num_classes=5)
    # state_dict = torch.load("variety_model.pt", map_location=torch.device('cpu'))
    # if all(key.startswith("module.") for key in state_dict.keys()):
    #     state_dict = {k[7:]: v for k, v in state_dict.items()}
    # variety_model.load_state_dict(state_dict, strict=False)
    # variety_model.eval()

    # # Sample image prediction
    # sample_image = Image.open("path/to/sample_image.jpg")
    # result = predict_variety(sample_image, variety_model)
    # print(f"Predicted variety: {result['name']} with confidence {result['confidence']:.2f}%")
    # print("Top predictions:", result['top_predictions'])

Epoch 1


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
